In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.191, 0.081, 0.067, 0.111],
    [0.082, 0.059, 0.063, 0.102],
    [0.036, 0.083, 0.076, 0.088],
    [0.029, 0.088, 0.235, 0.078],
    [0.306, 0.055, 0.113, 0.068],
    [0.038, 0.154, 0.033, 0.144],
    [0.014, 0.100, 0.047, 0.148],
    [0.029, 0.155, 0.139, 0.105],
    [0.014, 0.083, 0.150, 0.089], 
    [0.010, 0.142, 0.067, 0.069]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506])

# NORMALIZATION FUNCTION (min-max normalization)
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

# CODAS METHOD
def codas(matrix, weights, criteria_types, sites, criteria, tau=0.02):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    weighted_matrix = norm_matrix * weights

    # Negative-ideal solution (worst for each criterion)
    neg_ideal = np.min(weighted_matrix, axis=0)

    # Euclidean and Taxicab distances from negative-ideal
    euclidean = np.sqrt(((weighted_matrix - neg_ideal) ** 2).sum(axis=1))
    taxicab = np.abs(weighted_matrix - neg_ideal).sum(axis=1)

    # Assessment score
    # If difference in Euclidean distance between two alternatives < tau, use Taxicab distance as tie-breaker
    scores = np.zeros(len(sites))
    for i in range(len(sites)):
        count = 0
        for j in range(len(sites)):
            if i == j:
                continue
            diff = euclidean[i] - euclidean[j]
            if diff > tau:
                count += 1
            elif abs(diff) <= tau:
                if taxicab[i] > taxicab[j]:
                    count += 1
        scores[i] = count

    # Alternatively, for ranking, you can use the Euclidean distance directly (higher is better)
    df = pd.DataFrame({
        'Euclidean': euclidean,
        'Taxicab': taxicab,
        'CODAS Score': euclidean,  # For simple ranking, use Euclidean distance
        'Rank': rankdata(-euclidean, method='min')
    }, index=sites).sort_values('Rank')

    return df

result_df = codas(matrix, weights, criteria_types, sites, criteria)
print(result_df)


        Euclidean   Taxicab  CODAS Score  Rank
Site4    0.409899  0.525243     0.409899     1
Site8    0.407197  0.574414     0.407197     2
Site6    0.348099  0.373714     0.348099     3
Site10   0.312252  0.371330     0.312252     4
Site5    0.297679  0.409373     0.297679     5
Site9    0.247873  0.329879     0.247873     6
Site1    0.191765  0.313894     0.191765     7
Site7    0.160235  0.191080     0.160235     8
Site3    0.130912  0.204783     0.130912     9
Site2    0.086100  0.135203     0.086100    10
